In [8]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv("cleaned_data.csv")
df.drop(columns=df.columns.values[0:2], inplace=True)
df.dropna(inplace=True)
df.head()

/home/psachin/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,cleaned text,joke
0,soooo glad circumcised corona covered foreskin...,1.0
1,know google platform recording bowel movements...,1.0
2,difference wife vacuum vacuum nt snore sex amp...,1.0
3,french man say attractive teletubby oo lala,1.0
4,yo mama yo momma fat went zoo hippos got jealous,1.0


In [12]:
df_jokes = df[df['joke']>0]
df_non_jokes = df[df['joke']<0]
df = pd.concat([df_jokes.sample(25000, random_state=42),
                df_non_jokes.sample(25000, random_state=42)])
df.head()

,cleaned text,joke
544835,walked bar saw fat bird dancing table went sai...,1.0
120452,band antivaxxers refuse listen cure,1.0
100104,call building got engaged premrs,1.0
501226,shit gross one best friend dad used tell us me...,1.0
752947,hear couple stole calendar got six months,1.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned text'], df['joke'], test_size=0.2, random_state=42)

In [14]:
X_train = [doc.split() for doc in X_train]
X_test = [doc.split() for doc in X_test]

In [15]:
w2v = Word2Vec(vector_size=200, window=5, min_count=4, workers=8)
k = w2v.build_vocab(X_train)

w2v.train(X_train, total_examples=len(X_train), epochs=32)

vectors = w2v.wv

In [16]:
new_X_train = pd.DataFrame()

for words in X_train:
    temp = pd.DataFrame()
    for word in words:
        if word in vectors:
            word_vec=vectors[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
    
    doc_vector = temp.mean()
    new_X_train = new_X_train.append(doc_vector, ignore_index = True)

X_train = new_X_train
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.471662,-0.105823,0.141106,1.082060,0.435106,0.208837,0.343112,0.026890,-0.360837,0.339845,...,0.302998,0.136211,0.418451,-0.644343,-0.326274,-0.181561,-0.289597,0.332735,0.219044,0.097523
1,0.142114,0.079843,-0.133994,0.663394,0.202824,-0.144359,0.830120,-0.106749,-0.100603,0.193024,...,0.091185,0.277078,0.204967,-1.057089,0.116309,0.187670,-0.255678,-0.165519,-0.113628,-0.024045
2,0.025458,-0.053820,-0.272434,-0.376429,-0.023166,0.690617,-0.027804,0.264863,-0.042776,0.103287,...,-0.268477,-0.061476,-0.105062,-0.025746,-0.612047,-0.188620,-0.284875,0.252727,0.052159,-0.070231
3,-0.405058,-0.239134,0.008078,-0.016324,-0.096806,-0.202326,0.570183,-0.080581,-0.862294,-0.106916,...,0.165170,0.887379,-0.244643,0.125527,0.085894,0.431962,-0.124107,0.155835,0.209816,0.032092
4,0.347161,0.147152,-0.497897,-0.151911,0.295090,-0.495647,0.565214,-0.840417,-0.213575,0.333842,...,0.065079,0.488338,-0.795504,0.273271,0.658603,0.620847,-0.341721,-0.656953,0.013897,0.044868


In [17]:
new_X_test = pd.DataFrame()

for words in X_test:
    temp = pd.DataFrame()
    for word in words:
        if word in vectors:
            word_vec=vectors[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
    
    doc_vector = temp.mean()
    new_X_test = new_X_test.append(doc_vector, ignore_index = True)

X_test = new_X_test
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.514931,0.066924,0.283757,-0.342711,0.371650,-0.172469,0.662662,0.224183,0.186798,-0.531669,...,0.497229,-0.139081,0.007886,-0.006092,-0.152987,0.029164,0.012000,-0.363776,-0.441718,0.289116
1,-0.453635,-1.322479,-0.729486,-0.751255,0.098660,0.834732,0.665277,0.403482,-0.199655,-0.153378,...,-0.670086,0.598963,-0.537007,-0.054678,-0.617458,-0.031273,-1.331669,0.160255,-0.313897,-0.545228
2,0.233887,0.202393,0.424249,-0.047810,0.140363,-0.354333,0.675976,0.545283,-0.048028,0.047760,...,0.097640,0.580722,0.312917,-0.057497,0.198498,0.412545,0.437413,-0.018681,0.134740,0.283541
3,-0.039103,0.320994,0.002351,-0.444661,0.879878,-0.575506,1.032899,-0.738810,0.326447,-0.110939,...,0.159717,0.468021,0.239384,-0.115394,-0.385033,-0.220403,-0.529189,0.331265,-0.242739,-0.227001
4,-0.370635,-0.287394,-0.107264,0.246943,-0.351954,0.046553,0.213460,0.511254,0.179422,-0.100362,...,0.672469,0.110852,-0.273577,0.020335,0.254829,-0.491182,-0.300228,0.037091,-0.611585,-0.140147


In [ ]:
X_train['label'] = y_train
X_test['label'] = y_test

X_train.to_csv('Word2Vec train.csv')
X_test.to_csv('Word2Vec test.csv')